# 🚗 Low-Level Design of a Ride-Sharing App

Hey there! Ready to dive into designing a ride-sharing app like Careem or Bykea, tailored for cities like Karachi, Lahore, or Islamabad? 🚲 This project walks you through creating a modular, scalable system using **C++**, with UML diagrams and SOLID principles. We'll build features like ride booking, vehicle management, and trip tracking, with room for AI-driven features like fare prediction or route optimization. Let's make it happen! 💪

## Why Low-Level Design (LLD) for a Ride-Sharing App?

Ride-sharing apps are a big deal in Pakistan, solving real-world transport challenges in busy cities. A solid LLD ensures the system is modular, scalable, and ready for features like AI-based pricing or traffic-avoiding routes (e.g., dodging jams in Saddar, Karachi). This project is perfect for AI engineers looking to integrate machine learning with practical systems.

### Project Goals
- Allow users to book rides (bike or car).
- Manage vehicles (e.g., Mehran, Honda bike).
- Track ride status (e.g., 'Driver Assigned', 'Ride Completed').
- Save ride details to a file (scalable to databases).
- Design for scalability to add new vehicle types (e.g., rickshaws) or AI features.

### Tools Used
- **Language**: C++ (for this example).
- **Design Tool**: UML diagrams (Class, Sequence).
- **Storage**: File-based (scalable to MongoDB/PostgreSQL).
- **Future AI Integration**: TensorFlow for fare prediction, FastAPI for APIs.

## 1. Approach to LLD

We'll use a **Bottom-Up Approach**, starting with small components (vehicles, rides) and building up to the main app. This ensures modularity and scalability, perfect for interviews. Steps:
1. Create a bad design (one class doing everything).
2. Identify flaws using SOLID principles.
3. Refactor into a modular design with UML and code.

## 2. Bad Design

Let’s start with a flawed design where a single `RideSharingApp` class handles everything: vehicle management, ride booking, status tracking, and file saving. This violates SOLID principles like Single Responsibility Principle (SRP) and Open-Closed Principle (OCP).

### 2.1 Features
- Add vehicles (e.g., 'Honda Bike', 'Suzuki Mehran').
- Book a ride with a vehicle type and destination (e.g., 'Gulberg to Mall Road').
- Track ride status (e.g., 'Driver Assigned').
- Save ride details to a file.

### 2.2 UML Diagram (Bad Design)
```plaintext
+------------------+
|  RideSharingApp  |
+------------------+
| - vehicles: vector<string> |
| - rides: vector<string>    |
| - rideStatus: string       |
| + addVehicle()             |
| + bookRide()               |
| + trackRideStatus()        |
| + saveRideDetails()        |
+------------------+
```

### 2.3 Code (Bad Design)

In [ ]:
#include <iostream>
#include <vector>
#include <fstream>
#include <string>
using namespace std;

class RideSharingApp {
private:
    vector<string> vehicles; // Stores vehicle names
    vector<string> rides;   // Stores ride details (vehicle + destination)
    string rideStatus;      // Current ride status

public:
    void addVehicle(string vehicle) {
        vehicles.push_back(vehicle);
    }

    void bookRide(string vehicleType, string destination) {
        for (const auto& vehicle : vehicles) {
            if (vehicle == vehicleType) {
                rides.push_back(vehicleType + " to " + destination);
                rideStatus = "Driver Assigned";
                return;
            }
        }
        rideStatus = "Vehicle Not Found";
    }

    string trackRideStatus() {
        return rideStatus;
    }

    void saveRideDetails() {
        ofstream file("ride_details.txt");
        if (file.is_open()) {
            for (const auto& ride : rides) {
                file << ride << "\n";
            }
            file.close();
            cout << "Ride details saved successfully!" << endl;
        } else {
            cout << "Unable to open file!" << endl;
        }
    }
};

int main() {
    RideSharingApp app;
    app.addVehicle("Honda Bike");
    app.addVehicle("Suzuki Mehran");
    app.bookRide("Honda Bike", "Gulberg to Mall Road");
    cout << "Ride Status: " << app.trackRideStatus() << endl;
    app.saveRideDetails();
    return 0;
}


### 2.4 Output
```
Ride Status: Driver Assigned
Ride details saved successfully!
```
**File Content (ride_details.txt):**
```
Honda Bike to Gulberg to Mall Road
```

### 2.5 Issues with Bad Design
1. **SRP Violation**: `RideSharingApp` handles multiple responsibilities (vehicle management, ride booking, status tracking, file saving).
2. **OCP Violation**: Adding new vehicle types or statuses requires modifying the class.
3. **No Abstraction**: Uses strings for vehicles/rides, lacking polymorphism.
4. **Tightly Coupled**: Hardcoded logic limits extensibility (e.g., for database storage or AI features).
5. **Scalability Issues**: Can’t easily support new features like Urdu notifications or JazzCash payments.

## 3. Improved Design

Let’s refactor using SOLID principles for modularity and scalability. We’ll introduce polymorphism, separate responsibilities, and use abstractions.

### 3.1 Components
1. **Vehicle (Abstract Class)**: Defines vehicle types with `getDetails()`.
2. **BikeVehicle, CarVehicle**: Concrete vehicle implementations.
3. **Ride**: Manages ride details (vehicle, destination, status).
4. **Persistence (Abstract Class)**: Handles saving ride details.
5. **FileStorage, DBStorage**: Concrete storage implementations.
6. **RideManager**: Manages rides and vehicles.
7. **RideSharingApp**: Facade for client interaction.

### 3.2 UML Diagram (Improved Design)
```plaintext
+-----------------+
|     Vehicle     |
+-----------------+
| + getDetails(): string |
+-----------------+
          ^
          |
  +-------+-------+
  |               |
+-----------------+  +-----------------+
|   BikeVehicle   |  |   CarVehicle    |
+-----------------+  +-----------------+
| - name: string  |  | - name: string  |
| + getDetails(): string | | + getDetails(): string |
+-----------------+  +-----------------+

+-----------------+
|      Ride       |
+-----------------+
| - vehicle: Vehicle* |
| - destination: string |
| - status: string |
| + bookRide()     |
| + getStatus()    |
| + getDetails()   |
+-----------------+

+-----------------+
|   RideManager   |
+-----------------+
| - rides: vector<Ride*> |
| - vehicles: vector<Vehicle*> |
| + addVehicle()   |
| + bookRide()     |
| + getRideStatus()|
| + getRideDetails()|
+-----------------+

+-----------------+
|   Persistence   |
+-----------------+
| + save(content: string): void |
+-----------------+
          ^
          |
  +-------+-------+
  |               |
+-----------------+  +-----------------+
|   FileStorage   |  |    DBStorage    |
+-----------------+  +-----------------+
| + save(content: string): void | | + save(content: string): void |
+-----------------+  +-----------------+

+-----------------+
|  RideSharingApp |
+-----------------+
| - rideManager: RideManager* |
| - storage: Persistence* |
| + addVehicle()   |
| + bookRide()     |
| + trackRideStatus() |
| + saveRideDetails() |
+-----------------+
```

### 3.3 Code (Improved Design)

In [ ]:
#include <iostream>
#include <vector>
#include <fstream>
#include <string>
using namespace std;

// Abstract class for vehicles
class Vehicle {
public:
    virtual string getDetails() = 0;
    virtual ~Vehicle() {}
};

// Concrete bike vehicle
class BikeVehicle : public Vehicle {
private:
    string name;
public:
    BikeVehicle(string n) : name(n) {}
    string getDetails() override { return "Bike: " + name; }
};

// Concrete car vehicle
class CarVehicle : public Vehicle {
private:
    string name;
public:
    CarVehicle(string n) : name(n) {}
    string getDetails() override { return "Car: " + name; }
};

// Ride class to manage ride details
class Ride {
private:
    Vehicle* vehicle;
    string destination;
    string status;
public:
    Ride(Vehicle* v, string dest) : vehicle(v), destination(dest), status("Driver Assigned") {}
    void bookRide(Vehicle* v, string dest) {
        vehicle = v;
        destination = dest;
        status = "Driver Assigned";
    }
    string getStatus() { return status; }
    string getDetails() { return vehicle->getDetails() + " to " + destination + " (" + status + ")"; }
    ~Ride() { delete vehicle; }
};

// Abstract class for persistence
class Persistence {
public:
    virtual void save(string content) = 0;
    virtual ~Persistence() {}
};

// Concrete file storage
class FileStorage : public Persistence {
public:
    void save(string content) override {
        ofstream file("ride_details.txt");
        if (file.is_open()) {
            file << content;
            file.close();
            cout << "Ride details saved successfully!" << endl;
        } else {
            cout << "Unable to open file!" << endl;
        }
    }
};

// Concrete database storage (placeholder)
class DBStorage : public Persistence {
public:
    void save(string content) override {
        cout << "Saving to database: " << content << endl;
    }
};

// RideManager class to handle ride-related operations
class RideManager {
private:
    vector<Ride*> rides;
    vector<Vehicle*> vehicles;
public:
    void addVehicle(Vehicle* vehicle) {
        vehicles.push_back(vehicle);
    }
    void bookRide(string vehicleType, string destination) {
        for (auto vehicle : vehicles) {
            if (vehicle->getDetails().find(vehicleType) != string::npos) {
                rides.push_back(new Ride(vehicle, destination));
                return;
            }
        }
        cout << "Vehicle not found!" << endl;
    }
    string getRideStatus(int index) {
        if (index < rides.size()) {
            return rides[index]->getStatus();
        }
        return "Invalid ride index";
    }
    string getRideDetails() {
        string content = "";
        for (auto ride : rides) {
            content += ride->getDetails() + "\n";
        }
        return content;
    }
    ~RideManager() {
        for (auto ride : rides) delete ride;
        for (auto vehicle : vehicles) delete vehicle;
    }
};

// Main app class (Facade)
class RideSharingApp {
private:
    RideManager* rideManager;
    Persistence* storage;
public:
    RideSharingApp(RideManager* rm, Persistence* store) : rideManager(rm), storage(store) {}
    void addVehicle(Vehicle* vehicle) { rideManager->addVehicle(vehicle); }
    void bookRide(string vehicleType, string destination) { rideManager->bookRide(vehicleType, destination); }
    string trackRideStatus(int index) { return rideManager->getRideStatus(index); }
    void saveRideDetails() { storage->save(rideManager->getRideDetails()); }
    ~RideSharingApp() { delete rideManager; delete storage; }
};

int main() {
    RideManager* rm = new RideManager();
    Persistence* storage = new FileStorage();
    RideSharingApp app(rm, storage);

    app.addVehicle(new BikeVehicle("Honda Bike"));
    app.addVehicle(new CarVehicle("Suzuki Mehran"));
    app.bookRide("Honda Bike", "Gulberg to Mall Road");
    cout << "Ride Status: " << app.trackRideStatus(0) << endl;
    app.saveRideDetails();
    return 0;
}


### 3.4 Output
```
Ride Status: Driver Assigned
Ride details saved successfully!
```
**File Content (ride_details.txt):**
```
Bike: Honda Bike to Gulberg to Mall Road (Driver Assigned)
```

### 3.5 Why This Design is Better
1. **Single Responsibility Principle (SRP)**:
   - `Vehicle`: Manages vehicle details.
   - `Ride`: Handles ride booking and status.
   - `Persistence`: Manages storage.
   - `RideManager`: Manages rides and vehicles.
   - `RideSharingApp`: Acts as a facade for client interaction.
2. **Open-Closed Principle (OCP)**:
   - Add new vehicle types (e.g., `RickshawVehicle`) by extending `Vehicle`.
   - Add new storage (e.g., cloud storage) by extending `Persistence`.
3. **Liskov Substitution Principle (LSP)**:
   - `BikeVehicle` or `CarVehicle` can replace `Vehicle` in `Ride`.
4. **Interface Segregation Principle (ISP)**:
   - `Vehicle` has only `getDetails()`, `Persistence` has only `save()`.
5. **Dependency Inversion Principle (DIP)**:
   - `RideSharingApp` depends on `Persistence` abstraction, not `FileStorage`.

### 3.6 Sequence Diagram
```plaintext
Client -> RideSharingApp: addVehicle(new BikeVehicle("Honda Bike"))
RideSharingApp -> RideManager: addVehicle()
Client -> RideSharingApp: bookRide("Honda Bike", "Gulberg to Mall Road")
RideSharingApp -> RideManager: bookRide()
RideManager -> Vehicle: getDetails()
RideManager -> Ride: bookRide(vehicle, destination)
Client -> RideSharingApp: trackRideStatus(0)
RideSharingApp -> RideManager: getRideStatus()
RideManager -> Ride: getStatus()
Client -> RideSharingApp: saveRideDetails()
RideSharingApp -> RideManager: getRideDetails()
RideManager -> Ride: getDetails()
RideSharingApp -> Persistence: save(content)
```

## 4. Tips for Having Fun
- **Extend Vehicle Types**: Add a `RickshawVehicle` class to support rickshaws.
- **Add Statuses**: Update `Ride` to handle statuses like 'Ride Completed' or 'Cancelled'.
- **Database Integration**: Implement `DBStorage` with MongoDB or PostgreSQL.
- **AI Features**: Integrate TensorFlow for fare prediction based on distance and traffic.
- **Localization**: Add Urdu notifications for users in Pakistan.
- Check out [CppReference](https://en.cppreference.com/) for more C++ tricks!

## 5. What’s Next?
- Save this as a notebook in your learning path.
- Combine with AI projects (e.g., route optimization using reinforcement learning).
- Explore FastAPI for real-time ride updates or JazzCash for payments.